In [ ]:
# 13장 네트워크와 프로세스 간 통신 다루기
# -------------------------------------
# 하나의 컴퓨터에서 다른 컴퓨터와 데이터를 주고받으려면 통신이 필요하다. 
# 이번 장에서는 비동기, 소켓, 시그널과 관련된 통신 프로그램을 작성할 때 사용하는 파이썬 모듈을 알아본다.

## 071 비동기 방식으로 프로그래밍하려면? ― asyncio


In [ ]:
# 071 비동기 방식으로 프로그래밍하려면? ― asyncio

# asyncio는 async/await 구문을 사용하여 동시성 코드를 작성할 수 있게 해주는 모듈로, asyncio를 사용하면 단일 스레드 작업을 병렬로 처리할 수 있다.

# 파이썬 3.7 버전 이상부터 사용할 수 있다.

# 문제
# 다음은 서로 다른 입력 값으로 sum() 함수를 2번 수행하여 결괏값을 출력하는 파이썬 프로그램이다.

# import time

# def sleep():
#     time.sleep(1)

# def sum(name, numbers):
#     start = time.time()
#     total = 0
#     for number in numbers:
#         sleep()
#         total += number
#         print(f'작업중={name}, number={number}, total={total}')
#     end = time.time()
#     print(f'작업명={name}, 걸린시간={end-start}')
#     return total


# def main():
#     start = time.time()

#     result1 = sum("A", [1, 2])
#     result2 = sum("B", [1, 2, 3])

#     end = time.time()
#     print(f'총합={result1+result2}, 총시간={end-start}')


# if __name__ == "__main__":
#     main()

# sum() 함수에서 입력 값을 하나씩 더할 때마다 sleep() 함수에 의해 1초씩 시간이 소요된다. 이 프로그램을 실행한 결과는 다음과 같다.

# 작업중=A, number=1, total=1
# 작업중=A, number=2, total=3
# 작업명=A, 걸린시간=2.000162124633789
# 작업중=B, number=1, total=1
# 작업중=B, number=2, total=3
# 작업중=B, number=3, total=6
# 작업명=B, 걸린시간=3.0002427101135254
# 총합=9, 총시간=5.0004048347473145

# A 작업 2초, B 작업 3초 등 모두 5초의 시간이 걸리고 sum() 함수를 2번 호출한 결과의 총합은 9임을 알 수 있다.

# sum() 함수를 비동기 방식으로 호출하면 실행 시간을 줄일 수 있을 것이다. 
# 이 프로그램을 파이썬 비동기 모듈인 asyncio를 사용하여 비동기 방식으로 바꾸려면 어떻게 해야 할까?

In [1]:
import time

def sleep():
    time.sleep(1)
    
def sum(name, numbers):
    start = time.time()
    total = 0
    for number in numbers:
        sleep()
        total += number
        print(f'작업중={name}, number={number}, total={total}')
    end = time.time()
    print(f'작업명={name}, 걸린시간={end - start}')
    return total

def main():
    start = time.time()
    result1 = sum('A', [1,2])
    result2 = sum('B', [1,2,3])
    end = time.time()
    print(f'총합={result1+result2}, 총시간={end-start}')

if __name__ == '__main__':
    main()

작업중=A, number=1, total=1
작업중=A, number=2, total=3
작업명=A, 걸린시간=2.015300989151001
작업중=B, number=1, total=1
작업중=B, number=2, total=3
작업중=B, number=3, total=6
작업명=B, 걸린시간=3.026836633682251
총합=9, 총시간=5.042137622833252


In [ ]:
# 풀이
# 다음은 앞의 코드에 asyncio 모듈을 적용한 문제 풀이이다.

# [파일명: asyncio_sample.py]

# import asyncio
# import time


# async def sleep():
#     await asyncio.sleep(1)


# async def sum(name, numbers):
#     start = time.time()
#     total = 0
#     for number in numbers:
#         await sleep()
#         total += number
#         print(f'작업중={name}, number={number}, total={total}')
#     end = time.time()
#     print(f'작업명={name}, 걸린시간={end-start}')
#     return total


# async def main():
#     start = time.time()

#     task1 = asyncio.create_task(sum("A", [1, 2]))
#     task2 = asyncio.create_task(sum("B", [1, 2, 3]))

#     await task1
#     await task2

#     result1 = task1.result()
#     result2 = task2.result()

#     end = time.time()
#     print(f'총합={result1+result2}, 총시간={end-start}')


# if __name__ == "__main__":
#     asyncio.run(main())

# 함수를 비동기로 호출하려면 이렇게 def 앞에 async라는 키워드를 넣으면 된다. 
# 그러면 이제 이 함수는 비동기 함수가 된다. 이때 async를 적용한 비동기 함수를 코루틴이라 부른다.

# 또한, 코루틴 안에서 다른 코루틴을 호출할 때는 await sleep()과 같이 await를 함수명 앞에 붙여 호출해야 한다. 
# 코루틴 수행 중 await 코루틴을 만나면 await로 호출한 코루틴이 종료될 때까지 기다리지 않고 제어권을 메인 스레드나 다른 코루틴으로 넘긴다. 
# 이러한 방식을 넌블록킹(non-blocking)이라 한다. 그리고 호출한 코루틴이 종료되면 이벤트에 의해 다시 그 이후 작업이 수행된다.

# 여기서 하나 눈여겨봐야 할 점은 sleep() 함수에서 time.sleep(1) 대신 asyncio.sleep(1)를 사용한 부분이다. 
# 코루틴이 아닌 time.sleep(1)을 사용한다면 await가 적용되지 않아 실행 시간을 줄일 수 없다.

# main() 함수에서 사용한 asyncio.create_task()는 수행할 코루틴 작업(태스크)을 생성한다. 
# 여기서는 작업을 생성할 뿐이지 실제로 코루틴이 수행되는 것은 아니다. 
# 실제 코루틴 실행은 await 태스크가 담당한다. 그리고 실행 태스크의 결괏값은 태스크.result()로 얻을 수 있다.

# asyncio.create_task()는 코루틴을 동시에 실행하는 데 꼭 필요하다. 
# 다음처럼 태스크가 아닌 await로 코루틴을 실행한다면 코루틴이 동시에 실행되지 않고 하나씩 차례로 실행되어 이득이 없을 것이다.

# result1 = await sum("A", [1, 2])
# result2 = await sum("B", [1, 2, 3])

# asyncio.run(main())은 런 루프를 생성하여 main() 코루틴을 실행한다. 
# 코루틴을 실행하려면 런 루프가 반드시 필요하다. 코루틴이 모두 비동기적으로 실행되기 때문에 그 시작과 종료를 감지할 수 있는 이벤트 루프가 반드시 필요하기 때문이다.

# 이 코드를 실행한 결과는 다음과 같다.

# c:\projects\pylib>python asyncio_sample.py
# 작업중=A, number=1, total=1
# 작업중=B, number=1, total=1
# 작업중=A, number=2, total=3
# 작업명=A, 걸린시간=2.000617742538452
# 작업중=B, number=2, total=3
# 작업중=B, number=3, total=6
# 작업명=B, 걸린시간=3.000927209854126
# 총합=9, 총시간=3.000927209854126

# A 작업과 B 작업을 교대로 호출한다. (제어권이 await에 의해 계속 바뀐다는 것을 알 수 있다.) 
# 그리고 시간도 5초 걸리던 것이 3초만 걸리게 되므로 A, B 작업이 완전히 비동기적으로 동작했다는 것을 알 수 있다.

# 참고
# asyncio - 비동기 I/O: https://docs.python.org/ko/3/library/asyncio.html
# 코루틴과 태스크: https://docs.python.org/ko/3/library/asyncio-task.html

## 072 서버와 통신하는 게임을 만들려면? ― socket

In [ ]:
# 072 서버와 통신하는 게임을 만들려면? ― socket
# socket은 TCP 서버/클라이언트 프로그램을 작성할 때 사용하는 모듈이다.

# 문제
# 서버에서 1~9 사이의 숫자를 무작위로 생성하고 클라이언트가 접속하여 그 숫자를 맞추는 게임을 
# socket 모듈을 사용하여 프로그래밍하고자 한다. 어떻게 해야 할까?

# 숫자 맞추기 게임의 규칙은 다음과 같다.

# 1. 서버에서 1~9 사이의 숫자(정답)를 무작위로 생성하고 클라이언트의 접속을 기다린다.
# 2. 클라이언트는 서버에 접속하여 1~9 사이의 값을 입력하여 게임을 시작한다.
# 3. 서버는 클라이언트가 입력한 숫자가 정답보다 높을 때는 "너무 높아요"라고, 낮을 때는 "너무 낮아요"라고 응답한다.
# 4. 클라이언트가 0을 입력하면 "종료"라고 응답하고 서버를 종료한다.
# 5. 클라이언트가 정답을 입력하면 "정답"이라고 응답하고 서버를 종료한다.

In [ ]:
# 풀이
# 소켓 서버
# 숫자 게임 서버는 다음과 같이 만든다.

# [파일명: socket_server.py]

# import socket
# import random

# HOST = ''
# PORT = 50007

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('서버가 시작되었습니다.')
#     conn, addr = s.accept()
#     with conn:
#         answer = random.randint(1, 9)
#         print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
#         while True:
#             data = conn.recv(1024).decode('utf-8')
#             print(f'데이터:{data}')

#             try:
#                 n = int(data)
#             except ValueError:
#                 conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
#                 continue

#             if n == 0:
#                 conn.sendall(f"종료".encode('utf-8'))
#                 break
#             if n > answer:
#                 conn.sendall("너무 높아요".encode('utf-8'))
#             elif n < answer:
#                 conn.sendall("너무 낮아요".encode('utf-8'))
#             else:
#                 conn.sendall("정답".encode('utf-8'))
#                 break

# 먼저 socket.socket(socket.AF_INET, socket.SOCK_STREAM)으로 소켓 객체 s를 생성한다. 
# 첫 번째 매개변수 socket.AF_INET은 IPv4 인터넷 프로토콜을, 두 번째 매개변수 socket.SOCK_STREAM은 소켓 유형이 문자열 등을 주고받는 스트림 방식임을 의미한다.

# s.bind((HOST, PORT))는 소켓 서버가 HOST라는 IP 주소의 PORT 번호에 해당하는 포트로 연결되도록 설정한다는 뜻이다. 
# HOST에는 빈 문자열을 지정했으므로 외부 접속을 허용한다. 만약 HOST에 빈 문자열 대신 'localhost'로 설정한다면 로컬 접속만 허용한다.

# s.listen() 은 서버 소켓이 클라이언트와의 연결을 시작할 수 있도록 바인딩된 포트를 연다. conn, addr = s.accept()는 클라이언트가 접속하면 연결을 수락하고 conn, addr을 반환한다. 이때 conn은 서버와 클라이언트가 연결된 소켓을 의미하고 addr은 클라이언트의 접속 IP를 의미한다.

# 클라이언트가 전송한 값을 수신하려면 conn.recv() 함수를 사용하고 클라이언트에 값을 송신하려면 conn.sendall() 함수를 사용한다. 이때 주고받는 메시지는 바이트 문자열이어야 하므로 보낼 때는 UTF-8로 인코딩하고 받을 때에도 UTF-8로 디코딩해야 한다.

# 참고: 부록 - 01 파이썬과 유니코드

# conn.recv(1024)에서 1024는 한 번에 수신받을 데이터의 최대 바이트 수를 의미하며 보통 1킬로바이트를 의미하는 1,024바이트를 사용한다. 만약 클라이언트로부터 1,024바이트 이상의 데이터를 수신해야 한다면 더 큰 숫자를 사용하거나 루프를 사용하여 conn.recv() 함수를 여러 번 수행해야 한다.

# 이후 클라이언트가 정답 또는 0을 입력할 때까지 클라이언트와 데이터를 주고받는다.

In [2]:
%%writefile socket_server.py
# socket_server.py

import socket
import random

HOST = ''
PORT = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print('서버가 시작되었습니다.')
    conn, addr = s.accept()
    with conn:
        answer = random.randint(1, 9)
        print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
        while True:
            data = conn.recv(1024).decode('utf-8')
            print(f'데이터:{data}')

            try:
                n = int(data)
            except ValueError:
                conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
                continue

            if n == 0:
                conn.sendall(f"종료".encode('utf-8'))
                break
            if n > answer:
                conn.sendall("너무 높아요".encode('utf-8'))
            elif n < answer:
                conn.sendall("너무 낮아요".encode('utf-8'))
            else:
                conn.sendall("정답".encode('utf-8'))
                break

Writing socket_server.py


In [ ]:
# 소켓 클라이언트
# 숫자 게임 클라이언트는 다음과 같이 만든다.

# [파일명: socket_client.py]

# import socket

# HOST = 'localhost'
# PORT = 50007

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.connect((HOST, PORT))

#     while True:
#         n = input("1-9 사이의 숫자를 입력하세요(0은 게임포기):")
#         if not n.strip():
#             print("입력값이 잘못되었습니다.")
#             continue
#         s.sendall(n.encode('utf-8'))
#         data = s.recv(1024).decode('utf-8')
#         print(f'서버응답:{data}')
#         if data == "정답" or data == "종료":
#             break
# 소켓 클라이언트는 소켓 서버보다 간단하다. 가장 큰 차이점은 서버 소켓에서는 연결된 클라이언트를 의미하는 conn 객체를 통해 데이터를 주고받았다면 소켓 클라이언트에서는 서버와 연결을 의미하는 소켓 객체인 s 객체를 통해 데이터를 주고받는다는 점이다.

# 소켓 클라이언트는 서버가 아니므로 HOST = 'localhost'와 같이 접속할 HOST를 정확하게 입력해야 한다. 그리고 서버에 접속하려면 s.connect ((HOST, PORT))와 같이 connect() 함수를 통해 접속해야 한다.

# s.connect((HOST, PORT)) 함수에 전달하는 인수가 HOST, PORT가 아닌 (HOST, PORT)와 같은 튜플임에 주의하자.

# 그리고 서버 소켓과 데이터를 주고받고자 소켓 객체의 s.recv(), s.sendall() 함수를 사용한다. 사용 방법은 서버 소켓과 마찬가지이다.

# 동작확인
# 소켓 서버를 실행하고 나서 소켓 클라이언트를 실행하여 게임을 진행하고 동작을 확인해 보자.

# 서버소켓 실행 예

# c:\projects\pylib>python socket_server.py
# 서버가 시작되었습니다.
# 클라이언트가 접속했습니다:('127.0.0.1', 14761), 정답은 4 입니다.
# 데이터:1
# 데이터:9
# 데이터:5
# 데이터:3
# 데이터:4
# 클라이언트소켓 실행 예

# c:\projects\pylib>python socket_client.py
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):1
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):9
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):5
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):3
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):4
# 서버응답:정답
# 참고
# socket - 저수준 네트워킹 인터페이스: https://docs.python.org/ko/3/library/socket.html

In [8]:
%%writefile socket_client.py 
# socket_client.py
import socket

HOST = 'localhost'
PORT = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))

    while True:
        n = input("1-9 사이의 숫자를 입력하세요(0은 게임포기):")
        if not n.strip():
            print("입력값이 잘못되었습니다.")
            continue
        s.sendall(n.encode('utf-8'))
        data = s.recv(1024).decode('utf-8')
        print(f'서버응답:{data}')
        if data == "정답" or data == "종료":
            break

Overwriting socket_client.py


## 73 SSL로 서버와 통신하려면? ― ssl

In [ ]:
# 73 SSL로 서버와 통신하려면? ― ssl
# ssl은 socket 모듈로 작성한 서버/클라이언트에 공개 키 암호화 방식을 적용할 때 사용하는 모듈이다.

# SSL(Secure Socket Layer, 보안 소켓 레이어)은 네트워크로 연결된 컴퓨터 간에 인증되고 암호화된 링크를 설정하는 프로토콜이다.

# 알아두면 좋아요
# 공개키 암호화 방식이란?
# 공개 키와 비밀 키를 사용하여 암호화하는 방식이다. 비밀 키로 암호화한 데이터는 공개 키로만 복호화하고 공개 키로 암호화한 데이터는 비밀 키로만 복호화는 방식이다. 
# 말 그대로 공개 키는 누구에게나 공개한 키지만 공개 키로 암호화한 데이터는 오직 비밀 키를 가진 서버에서만 복호화할 수 있으므로 클라이언트가 서버로 
# 안전하게 메시지를 전송할 수 있도록 하는 암호화 방법이다.

# 문제
# 서버와의 통신에 보안을 강화하고자 한다. 
# 이에 이전 절에서 socket 모듈로 만들었던 숫자 맞추기 서버/클라이언트 프로그램에 ssl 모듈을 사용하여 공개 키 방식의 암호화를 적용하려면 어떻게 해야 할까?

# 참고 : 072 서버와 통신하는 게임을 만들려면? - socket


In [ ]:
# 풀이
# ssl 모듈을 사용하면 소켓 서버/클라이언트에 간단하게 공개 키 방식의 암호화를 적용할 수 있다. 하지만, ssl을 적용하려면 다음과 같은 파일이 필요하다.

# CA.key: 공개 키
# CA.pem: 공개 키로 만든 인증서(클라이언트에 제공해야 하는 인증서)
# server.key: 비밀 키(서버에만 있는 키)
# server.crt: 서버 인증서(서버에만 있어야 하는 인증서)

# 인증서 생성하기
# openssl을 사용하면 이러한 파일을 생성할 수 있다.

# 다음은 리눅스 환경에서 openssl을 사용하여 인증서를 생성하는 예제이다.

# 먼저 openssl을 실행한다.

# $ openssl
# OpenSSL>
# OpenSSL 프롬프트가 나타나면 먼저 다음처럼 genrsa -out CA.key 2048 명령으로 CA.key 파일을 생성한다.

# OpenSSL> genrsa -out CA.key 2048
# Generating RSA private key, 2048 bit long modulus (2 primes)
# ....+++++
# ......................................................+++++
# e is 65537 (0x010001)
# OpenSSL>
# 그리고 다음처럼 req -x509 -new -nodes -key CA.key -days 365 -out CA.pem 명령으로 CA.pem 파일을 생성한다. 국가 코드만 KR로 바꾸고 모든 항목을 비워서 생성하였다. CA.pem은 CA.key가 필요하며 유효 기간은 365일로 설정하였다.

# OpenSSL> req -x509 -new -nodes -key CA.key -days 365 -out CA.pem
# You are about to be asked to enter information that will be incorporated
# into your certificate request.
# What you are about to enter is what is called a Distinguished Name or a DN.
# There are quite a few fields but you can leave some blank
# For some fields there will be a default value,
# If you enter '.', the field will be left blank.
# -----
# Country Name (2 letter code) [AU]:KR
# State or Province Name (full name) [Some-State]:
# Locality Name (eg, city) []:
# Organization Name (eg, company) [Internet Widgits Pty Ltd]:
# Organizational Unit Name (eg, section) []:
# Common Name (e.g. server FQDN or YOUR name) []:
# Email Address []:
# OpenSSL>
# 만약 CA.pem 파일을 생성할 때 다음과 같은 오류가 발생한다면 /etc/ssl/openssl.cnf 파일의 RANDFILE 항목을 주석 처리하자.

# Can't load /home/ubuntu/.rnd into RNG
# 140351232827840:error:2406F079:random number generator:RAND_load_file:Cannot open file:../crypto/rand/randfile.c:88:Filename=/home/ubuntu/.rnd
# [파일명:/etc/ssl/openssl.cnf]

# (... 생략 ...)
# #RANDFILE               = $ENV::HOME/.rnd
# (... 생략 ...)
# 그리고 다음처럼 genrsa -out server.key 2048 명령으로 server.key 파일을 생성한다.

# OpenSSL> genrsa -out server.key 2048
# Generating RSA private key, 2048 bit long modulus (2 primes)
# .........................................................+++++
# ...............................................................................................................................................................................+++++
# e is 65537 (0x010001)
# OpenSSL>
# server.key 파일을 생성한 후 server.csr 파일을 생성하기 전에 다음처럼 반드시 openssl 프롬프트를 종료하고 다시 시작해야 한다.

# OpenSSL> quit
# ubuntu@ip-172-26-7-225:~/tmp$ openssl
# OpenSSL>
# openssl을 다시 시작하지 않고 server.crt 파일을 생성하려 하면 problem creating object tsa_policy1=1.2.3.4.1과 같은 오류가 발생한다.

# 이제 다음처럼 req -new -key server.key -out server.csr 명령으로 server.csr 파일을 생성한다. 
# 이 때 주의할 점은 Common Name 항목이다. 
# 여기서는 이 항목에 pylib라는 이름을 적었다. 이 항목은 클라이언트에서 서버에 접속할 때 필요한 이름이므로 빠뜨려서는 안 된다.

# OpenSSL> req -new -key server.key -out server.csr
# You are about to be asked to enter information that will be incorporated
# into your certificate request.
# What you are about to enter is what is called a Distinguished Name or a DN.
# There are quite a few fields but you can leave some blank
# For some fields there will be a default value,
# If you enter '.', the field will be left blank.
# -----
# Country Name (2 letter code) [AU]:KR
# State or Province Name (full name) [Some-State]:
# Locality Name (eg, city) []:
# Organization Name (eg, company) [Internet Widgits Pty Ltd]:
# Organizational Unit Name (eg, section) []:
# Common Name (e.g. server FQDN or YOUR name) []:pylib
# Email Address []:

# Please enter the following 'extra' attributes
# to be sent with your certificate request
# A challenge password []:
# An optional company name []:
# OpenSSL>
# 마지막으로 x509 -req -in server.csr -CA CA.pem -CAkey CA.key -CAcreateserial -out server.crt 명령으로 server.crt 파일을 생성한다.

# OpenSSL> x509 -req -in server.csr -CA CA.pem -CAkey CA.key -CAcreateserial -out server.crt
# Signature ok
# subject=C = KR, ST = Some-State, O = Internet Widgits Pty Ltd, CN = pylib
# Getting CA Private Key
# OpenSSL>

In [ ]:
# SSL 서버
# 필요한 인증서 파일을 모두 준비했다면 ssl 모듈을 적용한 소켓 서버는 다음과 같이 만들면 된다.

# [파일명: ssl_server.py]

# import socket
# import ssl
# import random

# HOST = ''
# PORT = 50007

# context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
# context.load_cert_chain('server.crt', 'server.key')

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
#     sock.bind((HOST, PORT))
#     sock.listen()
#     print('서버가 시작되었습니다.')

#     with context.wrap_socket(sock, server_side=True) as s:
#         conn, addr = s.accept()
#         with conn:
#             answer = random.randint(1, 9)
#             print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
#             while True:
#                 data = conn.recv(1024).decode('utf-8')
#                 print(f'데이터:{data}')

#                 try:
#                     n = int(data)
#                 except ValueError:
#                     conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
#                     continue

#                 if n == 0:
#                     conn.sendall(f"종료".encode('utf-8'))
#                     break
#                 if n > answer:
#                     conn.sendall("너무 높아요".encode('utf-8'))
#                 elif n < answer:
#                     conn.sendall("너무 낮아요".encode('utf-8'))
#                 else:
#                     conn.sendall("정답".encode('utf-8'))
#                     break
# 소켓 서버에 SSL을 적용하려면 다음처럼 context 객체를 먼저 생성해야 한다.

# context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
# 서버 소켓이므로 ssl.PROTOCOL_TLS_SERVER라는 인수를 전달하여 생성한다. 그리고 다음처럼 생성한 서버 인증서 파일과 서버 키 파일을 불러와야 한다.

# context.load_cert_chain('server.crt', 'server.key')
# 생성한 소켓은 context를 사용하여 다음처럼 감싸야 한다.

# with context.wrap_socket(sock, server_side=True) as s:
# 그 외 나머지 코드에는 변경 사항이 없다.

In [2]:
%%writefile ssl_server.py
# ssl_server.py

import socket
import ssl
import random

HOST = ''
PORT = 50007

context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
context.load_cert_chain('server.crt', 'server.key')
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    sock.bind((HOST, PORT))
    sock.listen()
    print('서버가 시작되었습니다.')
    
    with context.wrap_socket(sock, server_side=True) as s:
        conn, addr = s.accept()
        with conn:
            answer = random.randint(1, 9)
            print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer}입니다.')
            while True:
                data = conn.recv(1024).decode('utf-8')
                print(f'데이터:{data}')
                
                try:
                    n = int(data)
                except ValueError:
                    conn.sendall(f'입력값이 올바르지 않습니다.:{data}'.encode('utf-8'))
                    continue
                if n == 0:
                    conn.sendall(f'종료'.encode('utf-8'))
                    break
                if n > answer:
                    conn.sendall('너무 높아요'.encode('utf-8'))
                elif n < answer:
                    conn.sendall('너무 낮아요'.encode('utf-8'))
                else:
                    conn.sendall('정답'.encode('utf-8'))
                    break

Writing ssl_server.py


In [ ]:
# SSL 클라이언트
# ssl 모듈을 적용한 소켓 클라이언트는 다음과 같다.

# [파일명: ssl_client.py]

# import socket
# import ssl

# HOST = 'localhost'
# PORT = 50007

# context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
# context.load_verify_locations('CA.pem')

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
#     with context.wrap_socket(sock, server_hostname='pylib') as s:
#         s.connect((HOST, PORT))

#         while True:
#             n = input("1-9 사이의 숫자를 입력하세요(0은 게임포기):")
#             if not n.strip():
#                 print("입력값이 잘못되었습니다.")
#                 continue
#             s.sendall(n.encode('utf-8'))
#             data = s.recv(1024).decode('utf-8')
#             print(f'서버응답:{data}')
#             if data == "정답" or data == "종료":
#                 break
# 소켓 클라이언트에 SSL을 적용하려면 다음처럼 context 객체를 먼저 생성해야 한다.

# context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
# 클라이언트 소켓이므로 ssl.PROTOCOL_TLS_CLIENT라는 인수를 전달하여 생성한다. 그리고 다음처럼 클라이언트용 인증서를 읽어와야 한다.

# context.load_verify_locations('CA.pem')
# 생성한 소켓은 context를 사용하여 다음처럼 감싸야 한다. 여기서 사용한 server_hostname은 server.csr 파일을 생성할 때 등록한 호스트명(Common Name)과 같아야 한다.

# with context.wrap_socket(sock, server_hostname='pylib') as s:
# 그 외 나머지 코드에는 변경 사항이 없다.

# 동작확인
# 서버 소켓과 클라이언트 소켓에 SSL을 적용하고 동작을 확인해 보자. 기존 일반 소켓 방식과 똑같이 동작한다는 것을 알 수 있다. SSL을 적용하여 더 안전하게 암호화한 데이터를 송수신할 뿐이다.

# 서버소켓 실행 예제

# c:\projects\pylib>python ssl_server.py
# 서버가 시작되었습니다.
# 클라이언트가 접속했습니다:('127.0.0.1', 14761), 정답은 4 입니다.
# 데이터:1
# 데이터:9
# 데이터:5
# 데이터:3
# 데이터:4
# 클라이언트소켓 실행 예제

# c:\projects\pylib>python ssl_client.py
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):1
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):9
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):5
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):3
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):4
# 서버응답:정답
# 참고
# ssl - 소켓 객체용 TLS/SSL 래퍼: https://docs.python.org/ko/3/library/ssl.html

In [3]:
%%writefile ssl_client.py
# ssl_client.py

import socket
import ssl

HOST = '192.168.200.100'
PORT = 50007

context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
context.load_verify_locations('CA.pem')

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    with context.wrap_socket(sock, server_hostname='pylib') as s:
        s.connect((HOST, PORT))

        while True:
            n = input("1-9 사이의 숫자를 입력하세요(0은 게임포기):")
            if not n.strip():
                print("입력값이 잘못되었습니다.")
                continue
            s.sendall(n.encode('utf-8'))
            data = s.recv(1024).decode('utf-8')
            print(f'서버응답:{data}')
            if data == "정답" or data == "종료":
                break

Writing ssl_client.py


## 074 여러 명이 동시에 접속하려면? ― select

In [ ]:
# 074 여러 명이 동시에 접속하려면? ― select
# select는 소켓 프로그래밍에서 I/O 멀티플렉싱을 가능하게 하는 모듈이다.

# I/O 멀티플렉싱(multiplexing)이란 하나의 전송로로 여러 종류의 데이터를 송수신하는 방식을 말한다.

# 문제
# 다음은 socket 모듈 절에서 만든 '숫자 맞추기 게임' 소켓 서버이다.

# 참고: 072 서버와 통신하는 게임을 만들려면? - socket

# import socket
# import random

# HOST = ''
# PORT = 50007

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('서버가 시작되었습니다.')
#     conn, addr = s.accept()
#     with conn:
#         answer = random.randint(1, 9)
#         print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
#         while True:
#             data = conn.recv(1024).decode('utf-8')
#             print(f'데이터:{data}')

#             try:
#                 n = int(data)
#             except ValueError:
#                 conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
#                 continue

#             if n == 0:
#                 conn.sendall(f"종료".encode('utf-8'))
#                 break
#             if n > answer:
#                 conn.sendall("너무 높아요".encode('utf-8'))
#             elif n < answer:
#                 conn.sendall("너무 낮아요".encode('utf-8'))
#             else:
#                 conn.sendall("정답".encode('utf-8'))
#                 break
# 하지만, 이 프로그램에는 다음과 같은 문제가 있다.

# 클라이언트가 소켓 서버에 접속하여 게임을 진행한 후 접속을 종료하면 소켓 서버 역시 종료되어 더는 다른 클라이언트가 연결할 수 없다.
# 소켓 서버가 여러 클라이언트와 동시에 게임을 진행할 수 없다.
# 클라이언트 요청을 동시에 처리하여 한꺼번에 여러 명이 동시에 플레이할 수 있도록 기존 소켓 서버 프로그램을 수정하려면 어떻게 해야 할까?


In [ ]:
# 풀이
# 이 문제는 스레드를 사용해서 해결할 수도 있다. 하지만, 스레드 방식은 스레드 수가 늘어날 수록 시스템에 부담이므로 효율적이지 못하다. 
# select 방식은 처음에는 블록킹되어 있다가 특정 이벤트가 발생하면 그때 작동하는 방식이므로 스레드보다 효율적이다.

# select 모듈을 사용한 문제 풀이는 다음과 같다.

# [파일명:select_server.py]

# import socket
# import select
# import random

# HOST = ''
# PORT = 50007

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('서버가 시작되었습니다.')

#     readsocks = [s]
#     answers = {}

#     while True:
#         readables, writeables, excpetions = select.select(readsocks, [], [])
#         for sock in readables:
#             if sock == s:  # 신규 클라이언트 접속
#                 newsock, addr = s.accept()
#                 answer = random.randint(1, 9)
#                 print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
#                 readsocks.append(newsock)
#                 answers[newsock] = answer  # 클라이언트 별 정답 생성
#             else:  # 이미 접속한 클라이언트의 요청 (게임진행을 위한 요청)
#                 conn = sock
#                 data = conn.recv(1024).decode('utf-8')
#                 print(f'데이터:{data}')

#                 try:
#                     n = int(data)
#                 except ValueError:
#                     conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
#                     continue

#                 answer = answers.get(conn)
#                 if n == 0:
#                     conn.sendall(f"종료".encode('utf-8'))
#                     conn.close()
#                     readsocks.remove(conn)  # 클라이언트 접속 해제시 readsocks에서 제거
#                 if n > answer:
#                     conn.sendall("너무 높아요".encode('utf-8'))
#                 elif n < answer:
#                     conn.sendall("너무 낮아요".encode('utf-8'))
#                 else:
#                     conn.sendall("정답".encode('utf-8'))
#                     conn.close()
#                     readsocks.remove(conn)  # 클라이언트 접속 해제시 readsocks에서 제거
# 기존 소스 코드에서 달라진 부분만 살펴보자.

# select의 동작 원리 살펴보기
# 수정한 코드에서 가장 핵심 부분은 다음 문장이다.

# readables, writeables, excpetions = select.select(readsocks, [], [])
# select.select(readsocks, [], [])는 readsocks에 포함된 소켓에서 이벤트가 발생하는지 감시하는 역할을 한다. 
# 감시하다가 readsocks에 속한 소켓에 이벤트가 발생하면 이후 이 문장을 실행한다. 
# readables는 수신한 데이터를 가진 소켓을 의미하고 writeables는 블로킹되지 않고 데이터를 전송할 수 있는 소켓을, 
# exceptions는 예외 상황이 발생한 소켓을 의미한다. 그리고 readables, writeables, exceptions는 모두 여러 개의 소켓으로 리스트를 구성한다.

# 이 코드에서 select 모듈은 다음과 같이 동작한다.

# readsocks = [s]에 의해 최초 readsocks에는 서버 소켓만 있기 때문에 select는 이 서버 소켓에 클라이언트가 접속하는지 감시한다.
# 신규 클라이언트가 서버 소켓에 접속하면 readsocks에 신규 클라이언트의 소켓을 추가한다(readsocks.append(newsock)). 이렇게 되면 이제 readsocks에는 서버 소켓 외에 클라이언트 소켓도 포함된다.
# 신규 접속이 아니라 이미 접속한 클라이언트에서 데이터 요청 등의 이벤트가 발생하면 if sock == s:가 거짓(False)이 되어 해당 클라이언트와 숫자 게임을 진행한다.
# 이 소켓 서버에서 writeables, exceptions는 불필요하므로 사용하지 않았다.

# 클라이언트마다 다른 정답으로 게임하기
# 그리고 새로 접속할 때마다 숫자 게임의 정답을 바꾸고자 answers라는 딕셔너리를 사용했다. 이 딕셔너리의 키는 클라이언트 소켓이고 값은 정답을 의미하는 숫자이다.

# 동작 확인
# 서버 프로그램(select_server.py)을 실행하고 다음처럼 여러 개의 클라이언트로 동시에 접속해 보자.

# 서버

# c:\projects\pylib>python select_server.py
# 서버가 시작되었습니다.
# 클라이언트가 접속했습니다:('127.0.0.1', 1774), 정답은 4 입니다.
# 클라이언트가 접속했습니다:('127.0.0.1', 1775), 정답은 5 입니다.
# 데이터:3
# 데이터:9
# 데이터:1
# 데이터:7
# 데이터:4
# 데이터:2
# 데이터:5
# 클라이언트1

# c:\projects\pylib>python socket_client.py
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):3
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):9
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):4
# 서버응답:정답
# 클라이언트2

# c:\projects\pylib>python socket_client.py
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):1
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):7
# 서버응답:너무 높아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):2
# 서버응답:너무 낮아요
# 1-9 사이의 숫자를 입력하세요(0은 게임포기):5
# 서버응답:정답
# 둘 이상의 클라이언트가 동시에 접속해도 잘 동작하는 것을 확인할 수 있다. 또한, 클라이언트와 접속이 끝나더라도 서버는 종료되지 않는다.



In [ ]:
# 알아두면 좋아요
# 스레드 방식의 서버
# 다음은 이 문제를 threading 모듈로 풀이한 코드이다.

# 참고: 066 스레드를 이용하여 병렬로 처리하려면? - threading

# [파일명: threading_server.py]

# import socket
# import random
# import threading

# HOST = ''
# PORT = 50007


# def handle_client(conn, addr):
#     with conn:
#         answer = random.randint(1, 9)
#         print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
#         while True:
#             data = conn.recv(1024).decode('utf-8')
#             print(f'데이터:{data}')

#             try:
#                 n = int(data)
#             except ValueError:
#                 conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
#                 continue

#             if n == 0:
#                 conn.sendall(f"종료".encode('utf-8'))
#                 break
#             if n > answer:
#                 conn.sendall("너무 높아요".encode('utf-8'))
#             elif n < answer:
#                 conn.sendall("너무 낮아요".encode('utf-8'))
#             else:
#                 conn.sendall("정답".encode('utf-8'))
#                 break


# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('서버가 시작되었습니다.')

#     while True:
#         conn, addr = s.accept()
#         t = threading.Thread(target=handle_client, args=(conn, addr))
#         t.start()
# 클라이언트의 요청을 처리하고자 handle_client() 함수를 생성하고 클라이언트가 접속하면 이 함수를 스레드로 수행한다.

# 참고
# select - I/O 완료 대기: https://docs.python.org/ko/3/library/select.html

## 075 멀티플레이 게임 서버를 업그레이드하려면? ― selectors

In [ ]:
# 075 멀티플레이 게임 서버를 업그레이드하려면? ― selectors
# selectors는 select를 확장하여 고수준 I/O 멀티플렉싱을 가능하도록 한 모듈로, select 대신 사용하도록 권장하는 모듈이다.

# 문제
# 다음은 앞 절에서 select 모듈로 만든 '숫자 맞추기 게임'의 소켓 서버이다.

# 참고 : 074 여러 명이 동시에 접속하려면? - select

# 이 프로그램을 select 대신 selectors 모듈을 사용하도록 수정하려면 어떻게 해야 할까?

# import socket
# import select
# import random

# HOST = ''
# PORT = 50007

# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('서버가 시작되었습니다.')

#     readsocks = [s]
#     answers = {}

#     while True:
#         readables, writeables, excpetions = select.select(readsocks, [], [])
#         for sock in readables:
#             if sock == s:
#                 newsock, addr = s.accept()
#                 answer = random.randint(1, 9)
#                 print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')
#                 readsocks.append(newsock)
#                 answers[newsock] = answer
#             else:
#                 conn = sock
#                 data = conn.recv(1024).decode('utf-8')
#                 print(f'데이터:{data}')

#                 try:
#                     n = int(data)
#                 except ValueError:
#                     conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))
#                     continue

#                 answer = answers.get(conn)
#                 if n == 0:
#                     conn.sendall(f"종료".encode('utf-8'))
#                     conn.close()
#                     readsocks.remove(conn)
#                 if n > answer:
#                     conn.sendall("너무 높아요".encode('utf-8'))
#                 elif n < answer:
#                     conn.sendall("너무 낮아요".encode('utf-8'))
#                 else:
#                     conn.sendall("정답".encode('utf-8'))
#                     conn.close()
#                     readsocks.remove(conn)

In [ ]:

# 풀이
# 다음은 앞의 프로그램을 selctors 모듈로 변경한 코드이다.

# [파일명: selectors_server.py]

# import socket
# import selectors
# import random

# HOST = ''
# PORT = 50007

# sel = selectors.DefaultSelector()  # 최적의 Selector를 생성한다.
# answers = {}


# def accept_client(sock):
#     """ 서버 소켓에 클라이언트가 접속하면 호출된다. """
#     conn, addr = sock.accept()
#     answer = random.randint(1, 9)
#     answers[conn] = answer
#     sel.register(conn, selectors.EVENT_READ, game_client)  # 클라이언트 소켓을 등록한다.
#     print(f'클라이언트가 접속했습니다:{addr}, 정답은 {answer} 입니다.')


# def game_client(conn):
#     """ 클라이언트 소켓에 데이터가 수신되면 호출된다. """
#     data = conn.recv(1024).decode('utf-8')
#     print(f'데이터:{data}')
#     try:
#         n = int(data)
#         answer = answers.get(conn)
#         if n == 0:
#             conn.sendall(f"종료".encode('utf-8'))
#             sel.unregister(conn)
#             conn.close()
#         elif n > answer:
#             conn.sendall("너무 높아요".encode('utf-8'))
#         elif n < answer:
#             conn.sendall("너무 낮아요".encode('utf-8'))
#         else:
#             conn.sendall("정답".encode('utf-8'))
#             sel.unregister(conn)
#             conn.close()
#     except ValueError:
#         conn.sendall(f'입력값이 올바르지 않습니다:{data}'.encode('utf-8'))


# with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
#     s.bind((HOST, PORT))
#     s.listen()
#     print('서버가 시작되었습니다.')
#     sel.register(s, selectors.EVENT_READ, accept_client)  # 서버 소켓을 등록한다.

#     while True:
#         events = sel.select()  # 클라이언트의 접속 또는 접속된 클라이언트의 데이터 요청을 감시
#         for key, mask in events:
#             callback = key.data  # 실행할 함수
#             callback(key.fileobj)  # 이벤트가 발생한 소켓을 인수로 실행할 함수를 실행한다.

# 먼저 DefaultSelector 객체를 다음과 같이 선언했다.

# sel = selectors.DefaultSelector()
# selectors에는 EpollSelector, KqueueSelector 등의 여러 가지가 있는데, DefaultSelector는 해당 시스템에서 사용할 수 있는 최적의 Selector를 반환한다. 예를 들어 BSD 시스템이라면 KqueueSelector를 반환할 것이다.

# selectors 모듈은 DefaultSelector로 생성한 객체에 이벤트(실행할 함수)를 등록해야 하는 구조이므로 클라이언트의 접속을 처리하는 함수 accept_client()와 클라이언트와 게임을 진행하는 함수 game_client()를 작성했다.

# 그리고 accept_client() 함수에서 sel.register(conn, selectors.EVENT_READ, game_client)로 클라이언트 소켓에 데이터를 수신하면 game_client() 함수가 실행되도록 설정했다. 이벤트에는 다음과 같은 것이 있다.

# EVENT_READ: 읽기 이벤트
# EVENT_WRITE: 쓰기 이벤트
# 메인 로직에서는 sel.register(s, selectors.EVENT_READ, accept_client)로 클라이언트가 서버 소켓에 접속하면 accept_client() 함수를 실행하도록 설정했다.

# 그리고 events = sel.select()로 등록한 소켓의 이벤트를 감시하게 했다. 만약 서버 소켓에 신규 클라이언트가 접속하면 콜백 함수로 등록한 accept_client() 함수가 실행될 것이고 등록된 클라이언트 소켓에 데이터가 수신되면 game_client() 함수가 실행될 것이다.

# events = sel.select() 코드에서 events의 key 속성은 다음과 같다.

# key.data: sel.register로 등록한 콜백함수
# key.fileobj: 이벤트가 발생한 소켓
# 발생한 events의 mask는 EVENT_READ 또는 EVENT_WRITE를 의미하는 숫자인데 여기서는 불필요하므로 사용하지 않았다.

# 참고
# selectors - 고수준 I/O 다중화: https://docs.python.org/ko/3/library/selectors.html

## 076 사용자가 보낸 신호를 처리하려면? ― signal


In [ ]:
# 076 사용자가 보낸 신호를 처리하려면? ― signal
# signal은 특정 신호를 수신했을 때 사용자가 정의한 함수를 호출하도록 한 모듈이다.

# 문제
# 다음은 사용자의 입력을 기다린다는 의미로 10초에 한 번씩 대기 중...이라는 메시지를 출력하는 프로그램이다.

# import time

# while True:
#     print('대기중...')
#     time.sleep(10)
# 그러나 프로그램 실행 중 Ctrl+C를 입력하면 키보드 인터럽트 오류가 발생하여 다음처럼 프로그램이 중단된다.

# 대기중...
# 대기중...
# Traceback (most recent call last):
#   File "c:\projects\pylib\signal_test.py", line 18, in <module>
#     time.sleep(10)
# KeyboardInterrupt
# ^C
# 사용자가 실수로 또는 고의로 Ctrl+C를 입력하더라도 프로그램이 중단되지 않도록 하려면 어떻게 해야 할까?

# 풀이
# Ctrl+C와 같은 키보드 인터럽트(SIGINT) 신호를 감지하고 다르게 동작하도록 하려면 signal 모듈을 사용해야 한다.

# 다음은 signal 모듈을 사용한 문제 풀이이다.

# [파일명: signal_sample.py]

# import time
# import signal


# def handler(signum, frame):
#     print("Ctrl+C 신호를 수신했습니다.")


# signal.signal(signal.SIGINT, handler)

# while True:
#     print('대기중...')
#     time.sleep(10)

# 이렇게 수정한 다음 프로그램을 실행하면 Ctrl+C를 입력해도 프로그램을 종료하지 않고 다음처럼 "Ctrl+C 신호를 수신했습니다."라는 문자열을 출력하는 것을 확인할 수 있다.

# c:\projects\pylib>python signal_sample.py
# 대기중...
# 대기중...
# Ctrl+C 신호를 수신했습니다.
# 이 프로그램의 중지하려면 Ctrl+Break 키를 입력하면 된다.

# Ctrl+C는 키보드 인터럽트(SIGINT)에 해당한다. 따라서 signal.signal(signal.SIGINT, handler)처럼 SIGINT 신호가 발생할 때는 
# 기본 동작을 무시하고 handler() 함수를 실행하도록 설정했다. 
# handler() 함수의 signum은 발생한 신호의 숫자 값이고 frame은 프로그램을 실행한 스택 프레임이다.

# 참고
# signal - 비동기 이벤트에 대한 처리기 설정: https://docs.python.org/ko/3/library/signal.html